### This is a very simple example on how to use PySpark ML on kaggle's titanic dataset.

Problem statement and dataset can be found here https://www.kaggle.com/c/titanic

### Prerequisites:
  * Basics understand of python https://www.kaggle.com/learn/python
  * Pandas https://www.kaggle.com/learn/pandas
  * Machine learning https://www.kaggle.com/learn/machine-learning

  * Apache Spark 

      https://docs.databricks.com/spark/latest/gentle-introduction/gentle-intro.html
      
      https://docs.databricks.com/spark/latest/gentle-introduction/gentle-intro.html#gentle-introduction-to-apache-spark
      
      https://docs.databricks.com/spark/latest/gentle-introduction/for-data-scientists.html

#### Importing the necessary libraries

In [4]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import QuantileDiscretizer


#### Starting Point: SparkSession

The entry point into all functionality in Spark is the SparkSession class. To create a basic SparkSession, just use SparkSession.builder

In [7]:
spark = SparkSession \
    .builder \
    .appName("Spark ML example on titanic data ") \
    .getOrCreate()

Next, we have to import the dataset. In this file, I am importing dataset from S3(https://docs.databricks.com/spark/latest/data-sources/aws/amazon-s3.html).

You can can directly upload dataset to databricks cloud(https://docs.databricks.com/user-guide/importing-data.html#import-data)

In [9]:
s3_bucket_path = "/mnt/lp-dataset/titanic/train.csv"

In [10]:
titanic_df = spark.read.csv(s3_bucket_path,header = 'True',inferSchema='True')

In [11]:
passengers_count = titanic_df.count()

In [12]:
print(passengers_count)

891

Lets  view few rows

In [14]:
titanic_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
PassengerId|Survived|Pclass| Name| Sex| Age|SibSp|Parch| Ticket| Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
 1| 0| 3|Braund, Mr. Owen ...| male|22.0| 1| 0| A/5 21171| 7.25| null| S|
 2| 1| 1|Cumings, Mrs. Joh...|female|38.0| 1| 0| PC 17599|71.2833| C85| C|
 3| 1| 3|Heikkinen, Miss. ...|female|26.0| 0| 0|STON/O2. 3101282| 7.925| null| S|
 4| 1| 1|Futrelle, Mrs. Ja...|female|35.0| 1| 0| 113803| 53.1| C123| S|
 5| 0| 3|Allen, Mr. Willia...| male|35.0| 0| 0| 373450| 8.05| null| S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 5 rows

Summary of data

In [16]:
titanic_df.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
summary| PassengerId| Survived| Pclass| Name| Sex| Age| SibSp| Parch| Ticket| Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
 count| 891| 891| 891| 891| 891| 714| 891| 891| 891| 891| 204| 889|
 mean| 446.0| 0.3838383838383838| 2.308641975308642| null| null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.2042079685746| null| null|
 stddev|257.3538420152301|0.48659245426485753|0.8360712409770491| null| null|14.526497332334035|1.1027434322934315| 0.8060572211299488|471609.26868834975|49.69342859718089| null| null|
 min| 1| 0| 1|"Andersson, Mr. A...|female| 0.42| 0| 0| 110152| 0.0| A10| C|
 max| 891| 1| 3|van Melkebeke, Mr...| male| 80.0| 8| 6| WE/P 5735| 512.3292| T| S|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+

Let's see Schema of our dataset

In [18]:
titanic_df.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- Embarked: string (nullable = true)

Let's select few features

In [20]:
titanic_df.select("Survived","Pclass","Embarked").show()

+--------+------+--------+
Survived|Pclass|Embarked|
+--------+------+--------+
 0| 3| S|
 1| 1| C|
 1| 3| S|
 1| 1| S|
 0| 3| S|
 0| 3| Q|
 0| 1| S|
 0| 3| S|
 1| 3| S|
 1| 2| C|
 1| 3| S|
 1| 1| S|
 0| 3| S|
 0| 3| S|
 0| 3| S|
 1| 2| S|
 0| 3| Q|
 1| 2| S|
 0| 3| S|
 1| 3| C|
+--------+------+--------+
only showing top 20 rows

### Let's do some simple exploratory data analysis (EDA)

How many Passengers Survived ?

In [23]:
titanic_df.groupBy("Survived").count().show()

+--------+-----+
Survived|count|
+--------+-----+
 1| 342|
 0| 549|
+--------+-----+

Out of 891 passengers in dataset, only around 342 survived.

We need to dig down more to get better insights from the data and see which categories of the passengers did survive and who didn't. We will try to check the survival rate by using the different features of the dataset. Some of the features being Sex, Port Of Embarcation, Age,etc.

Lets check survival rate with Sex

In [27]:
titanic_df.groupBy("Sex","Survived").count().show()

+------+--------+-----+
 Sex|Survived|count|
+------+--------+-----+
 male| 0| 468|
female| 1| 233|
female| 0| 81|
 male| 1| 109|
+------+--------+-----+

This looks interesting. The number of men on the ship is lot more than the number of women. Still the number of women saved is almost twice the number of males saved.

In [29]:
titanic_df.groupBy("Pclass","Survived").count().show()

+------+--------+-----+
Pclass|Survived|count|
+------+--------+-----+
 1| 0| 80|
 3| 1| 119|
 1| 1| 136|
 2| 1| 87|
 2| 0| 97|
 3| 0| 372|
+------+--------+-----+

We can clearly see that Passenegers Of Pclass 1 were given a very high priority while rescue. Even though the the number of Passengers in Pclass 3 were a lot higher, still the number of survival from them is very low.

#### Checking Null values

In [32]:
# This function use to print feature with null values and null count 
def null_value_count(df):
  null_columns_counts = []
  numRows = df.count()
  for k in df.columns:
    nullRows = df.where(col(k).isNull()).count()
    if(nullRows > 0):
      temp = k,nullRows
      null_columns_counts.append(temp)
  return(null_columns_counts)

In [33]:
# Calling function
null_columns_count_list = null_value_count(titanic_df)


In [34]:
spark.createDataFrame(null_columns_count_list, ['Column_With_Null_Value', 'Null_Values_Count']).show()

+----------------------+-----------------+
Column_With_Null_Value|Null_Values_Count|
+----------------------+-----------------+
 Age| 177|
 Cabin| 687|
 Embarked| 2|
+----------------------+-----------------+

Age feature has 177 null values.

In [36]:
mean_age = titanic_df.select(mean('Age')).collect()[0][0]
print(mean_age)

29.6991176471

To replace these NaN values, we can assign them the mean age of the dataset.But the problem is, there were many people with many different ages. We just cant assign a 4 year kid with the mean age that is 29 years.

we can check the Name feature. Looking upon the feature, we can see that the names have a salutation like Mr or Mrs. Thus we can assign the mean values of Mr and Mrs to the respective groups

In [39]:
titanic_df = titanic_df.withColumn("Initial",regexp_extract(col("Name"),"([A-Za-z]+)\.",1))

Using the Regex ""[A-Za-z]+)." we extract the initials from the Name. It looks for strings which lie between A-Z or a-z and followed by a .(dot).

In [41]:
titanic_df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+
PassengerId|Survived|Pclass| Name| Sex| Age|SibSp|Parch| Ticket| Fare|Cabin|Embarked|Initial|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+
 1| 0| 3|Braund, Mr. Owen ...| male|22.0| 1| 0| A/5 21171| 7.25| null| S| Mr|
 2| 1| 1|Cumings, Mrs. Joh...|female|38.0| 1| 0| PC 17599|71.2833| C85| C| Mrs|
 3| 1| 3|Heikkinen, Miss. ...|female|26.0| 0| 0|STON/O2. 3101282| 7.925| null| S| Miss|
 4| 1| 1|Futrelle, Mrs. Ja...|female|35.0| 1| 0| 113803| 53.1| C123| S| Mrs|
 5| 0| 3|Allen, Mr. Willia...| male|35.0| 0| 0| 373450| 8.05| null| S| Mr|
 6| 0| 3| Moran, Mr. James| male|null| 0| 0| 330877| 8.4583| null| Q| Mr|
 7| 0| 1|McCarthy, Mr. Tim...| male|54.0| 0| 0| 17463|51.8625| E46| S| Mr|
 8| 0| 3|Palsson, Master. ...| male| 2.0| 3| 1| 349909| 21.075| null| S| Master|
 9| 1| 3|Johnson, Mrs. Osc...|female|27.0| 0| 2| 347742|11.1333| null| S| Mrs|
 10| 1| 2|Nasser, Mrs. Nich...|female|14.0| 1| 0| 237736|30.0708| null| C| Mrs|
 11| 1| 3|Sandstrom, Miss. ...|female| 4.0| 1| 1| PP 9549| 16.7| G6| S| Miss|
 12| 1| 1|Bonnell, Miss. El...|female|58.0| 0| 0| 113783| 26.55| C103| S| Miss|
 13| 0| 3|Saundercock, Mr. ...| male|20.0| 0| 0| A/5. 2151| 8.05| null| S| Mr|
 14| 0| 3|Andersson, Mr. An...| male|39.0| 1| 5| 347082| 31.275| null| S| Mr|
 15| 0| 3|Vestrom, Miss. Hu...|female|14.0| 0| 0| 350406| 7.8542| null| S| Miss|
 16| 1| 2|Hewlett, Mrs. (Ma...|female|55.0| 0| 0| 248706| 16.0| null| S| Mrs|
 17| 0| 3|Rice, Master. Eugene| male| 2.0| 4| 1| 382652| 29.125| null| Q| Master|
 18| 1| 2|Williams, Mr. Cha...| male|null| 0| 0| 244373| 13.0| null| S| Mr|
 19| 0| 3|Vander Planke, Mr...|female|31.0| 1| 0| 345763| 18.0| null| S| Mrs|
 20| 1| 3|Masselmani, Mrs. ...|female|null| 0| 0| 2649| 7.225| null| C| Mrs|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+
only showing top 20 rows

In [42]:
titanic_df.select("Initial").distinct().show()


+--------+
 Initial|
+--------+
 Don|
 Miss|
Countess|
 Col|
 Rev|
 Lady|
 Master|
 Mme|
 Capt|
 Mr|
 Dr|
 Mrs|
 Sir|
Jonkheer|
 Mlle|
 Major|
 Ms|
+--------+

There are some misspelled Initials like Mlle or Mme that stand for Miss. I will replace them with Miss and same thing for other values.

In [44]:
titanic_df = titanic_df.replace(['Mlle','Mme', 'Ms', 'Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
               ['Miss','Miss','Miss','Mr','Mr',  'Mrs',  'Mrs',  'Other',  'Other','Other','Mr','Mr','Mr'])


In [45]:
titanic_df.select("Initial").distinct().show()


+-------+
Initial|
+-------+
 Miss|
 Other|
 Master|
 Mr|
 Mrs|
+-------+

lets check the average age by Initials

In [47]:
titanic_df.groupby('Initial').avg('Age').collect()

Out[ 360 ]: 
[Row(Initial=u'Miss', avg(Age)=21.86),
 Row(Initial=u'Other', avg(Age)=45.888888888888886),
 Row(Initial=u'Master', avg(Age)=4.574166666666667),
 Row(Initial=u'Mr', avg(Age)=32.73960880195599),
 Row(Initial=u'Mrs', avg(Age)=35.981818181818184)]

Let's impute missing values in age feature based on average age of Initials

In [49]:
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Initial"] == "Miss") & (titanic_df["Age"].isNull()), 22).otherwise(titanic_df["Age"]))
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Initial"] == "Other") & (titanic_df["Age"].isNull()), 46).otherwise(titanic_df["Age"]))
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Initial"] == "Master") & (titanic_df["Age"].isNull()), 5).otherwise(titanic_df["Age"]))
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Initial"] == "Mr") & (titanic_df["Age"].isNull()), 33).otherwise(titanic_df["Age"]))
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Initial"] == "Mrs") & (titanic_df["Age"].isNull()), 36).otherwise(titanic_df["Age"]))


Check the imputation

In [51]:
titanic_df.filter(titanic_df.Age==46).select("Initial").show()


+-------+
Initial|
+-------+
 Mr|
 Mr|
 Mr|
+-------+

In [52]:
titanic_df.select("Age").show()

+----+
 Age|
+----+
22.0|
38.0|
26.0|
35.0|
35.0|
33.0|
54.0|
 2.0|
27.0|
14.0|
 4.0|
58.0|
20.0|
39.0|
14.0|
55.0|
 2.0|
33.0|
31.0|
36.0|
+----+
only showing top 20 rows

Embarked feature has only two missining values. Let's check values within Embarked

In [54]:
titanic_df.groupBy("Embarked").count().show()

+--------+-----+
Embarked|count|
+--------+-----+
 Q| 77|
 null| 2|
 C| 168|
 S| 644|
+--------+-----+

Majority Passengers boarded from "S". We can impute with "S"

In [56]:
titanic_df = titanic_df.na.fill({"Embarked" : 'S'})


We can drop Cabin features as it has lots of null values

In [58]:
titanic_df = titanic_df.drop("Cabin")

In [59]:
titanic_df.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Embarked: string (nullable = false)
-- Initial: string (nullable = true)

We can create a new feature called "Family_size" and "Alone" and analyse it. This feature is the summation of Parch(parents/children) and SibSp(siblings/spouses). It gives us a combined data so that we can check if survival rate have anything to do with family size of the passengers

In [61]:
titanic_df = titanic_df.withColumn("Family_Size",col('SibSp')+col('Parch'))

In [62]:
titanic_df.groupBy("Family_Size").count().show()

+-----------+-----+
Family_Size|count|
+-----------+-----+
 1| 161|
 6| 12|
 3| 29|
 5| 22|
 4| 15|
 7| 6|
 10| 7|
 2| 102|
 0| 537|
+-----------+-----+

In [63]:
titanic_df = titanic_df.withColumn('Alone',lit(0))


In [64]:
titanic_df = titanic_df.withColumn("Alone",when(titanic_df["Family_Size"] == 0, 1).otherwise(titanic_df["Alone"]))

In [65]:
titanic_df.columns

Out[ 375 ]: 
['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Embarked',
 'Initial',
 'Family_Size',
 'Alone']

Lets convert Sex, Embarked & Initial columns from string to number using StringIndexer

In [67]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(titanic_df) for column in ["Sex","Embarked","Initial"]]
pipeline = Pipeline(stages=indexers)
titanic_df = pipeline.fit(titanic_df).transform(titanic_df)

In [68]:
titanic_df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+-------+-----------+-----+---------+--------------+-------------+
PassengerId|Survived|Pclass| Name| Sex| Age|SibSp|Parch| Ticket| Fare|Embarked|Initial|Family_Size|Alone|Sex_index|Embarked_index|Initial_index|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+-------+-----------+-----+---------+--------------+-------------+
 1| 0| 3|Braund, Mr. Owen ...| male|22.0| 1| 0| A/5 21171| 7.25| S| Mr| 1| 0| 0.0| 0.0| 0.0|
 2| 1| 1|Cumings, Mrs. Joh...|female|38.0| 1| 0| PC 17599|71.2833| C| Mrs| 1| 0| 1.0| 1.0| 2.0|
 3| 1| 3|Heikkinen, Miss. ...|female|26.0| 0| 0|STON/O2. 3101282| 7.925| S| Miss| 0| 1| 1.0| 0.0| 1.0|
 4| 1| 1|Futrelle, Mrs. Ja...|female|35.0| 1| 0| 113803| 53.1| S| Mrs| 1| 0| 1.0| 0.0| 2.0|
 5| 0| 3|Allen, Mr. Willia...| male|35.0| 0| 0| 373450| 8.05| S| Mr| 0| 1| 0.0| 0.0| 0.0|
 6| 0| 3| Moran, Mr. James| male|33.0| 0| 0| 330877| 8.4583| Q| Mr| 0| 1| 0.0| 2.0| 0.0|
 7| 0| 1|McCarthy, Mr. Tim...| male|54.0| 0| 0| 17463|51.8625| S| Mr| 0| 1| 0.0| 0.0| 0.0|
 8| 0| 3|Palsson, Master. ...| male| 2.0| 3| 1| 349909| 21.075| S| Master| 4| 0| 0.0| 0.0| 3.0|
 9| 1| 3|Johnson, Mrs. Osc...|female|27.0| 0| 2| 347742|11.1333| S| Mrs| 2| 0| 1.0| 0.0| 2.0|
 10| 1| 2|Nasser, Mrs. Nich...|female|14.0| 1| 0| 237736|30.0708| C| Mrs| 1| 0| 1.0| 1.0| 2.0|
 11| 1| 3|Sandstrom, Miss. ...|female| 4.0| 1| 1| PP 9549| 16.7| S| Miss| 2| 0| 1.0| 0.0| 1.0|
 12| 1| 1|Bonnell, Miss. El...|female|58.0| 0| 0| 113783| 26.55| S| Miss| 0| 1| 1.0| 0.0| 1.0|
 13| 0| 3|Saundercock, Mr. ...| male|20.0| 0| 0| A/5. 2151| 8.05| S| Mr| 0| 1| 0.0| 0.0| 0.0|
 14| 0| 3|Andersson, Mr. An...| male|39.0| 1| 5| 347082| 31.275| S| Mr| 6| 0| 0.0| 0.0| 0.0|
 15| 0| 3|Vestrom, Miss. Hu...|female|14.0| 0| 0| 350406| 7.8542| S| Miss| 0| 1| 1.0| 0.0| 1.0|
 16| 1| 2|Hewlett, Mrs. (Ma...|female|55.0| 0| 0| 248706| 16.0| S| Mrs| 0| 1| 1.0| 0.0| 2.0|
 17| 0| 3|Rice, Master. Eugene| male| 2.0| 4| 1| 382652| 29.125| Q| Master| 5| 0| 0.0| 2.0| 3.0|
 18| 1| 2|Williams, Mr. Cha...| male|33.0| 0| 0| 244373| 13.0| S| Mr| 0| 1| 0.0| 0.0| 0.0|
 19| 0| 3|Vander Planke, Mr...|female|31.0| 1| 0| 345763| 18.0| S| Mrs| 1| 0| 1.0| 0.0| 2.0|
 20| 1| 3|Masselmani, Mrs. ...|female|36.0| 0| 0| 2649| 7.225| C| Mrs| 0| 1| 1.0| 1.0| 2.0|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+-------+-----------+-----+---------+--------------+-------------+
only showing top 20 rows

In [69]:
titanic_df.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Embarked: string (nullable = false)
-- Initial: string (nullable = true)
-- Family_Size: integer (nullable = true)
-- Alone: integer (nullable = false)
-- Sex_index: double (nullable = false)
-- Embarked_index: double (nullable = false)
-- Initial_index: double (nullable = false)

Drop columns which are not required

In [71]:
titanic_df = titanic_df.drop("PassengerId","Name","Ticket","Cabin","Embarked","Sex","Initial")

In [72]:
titanic_df.show()

+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+
Survived|Pclass| Age|SibSp|Parch| Fare|Family_Size|Alone|Sex_index|Embarked_index|Initial_index|
+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+
 0| 3|22.0| 1| 0| 7.25| 1| 0| 0.0| 0.0| 0.0|
 1| 1|38.0| 1| 0|71.2833| 1| 0| 1.0| 1.0| 2.0|
 1| 3|26.0| 0| 0| 7.925| 0| 1| 1.0| 0.0| 1.0|
 1| 1|35.0| 1| 0| 53.1| 1| 0| 1.0| 0.0| 2.0|
 0| 3|35.0| 0| 0| 8.05| 0| 1| 0.0| 0.0| 0.0|
 0| 3|33.0| 0| 0| 8.4583| 0| 1| 0.0| 2.0| 0.0|
 0| 1|54.0| 0| 0|51.8625| 0| 1| 0.0| 0.0| 0.0|
 0| 3| 2.0| 3| 1| 21.075| 4| 0| 0.0| 0.0| 3.0|
 1| 3|27.0| 0| 2|11.1333| 2| 0| 1.0| 0.0| 2.0|
 1| 2|14.0| 1| 0|30.0708| 1| 0| 1.0| 1.0| 2.0|
 1| 3| 4.0| 1| 1| 16.7| 2| 0| 1.0| 0.0| 1.0|
 1| 1|58.0| 0| 0| 26.55| 0| 1| 1.0| 0.0| 1.0|
 0| 3|20.0| 0| 0| 8.05| 0| 1| 0.0| 0.0| 0.0|
 0| 3|39.0| 1| 5| 31.275| 6| 0| 0.0| 0.0| 0.0|
 0| 3|14.0| 0| 0| 7.8542| 0| 1| 1.0| 0.0| 1.0|
 1| 2|55.0| 0| 0| 16.0| 0| 1| 1.0| 0.0| 2.0|
 0| 3| 2.0| 4| 1| 29.125| 5| 0| 0.0| 2.0| 3.0|
 1| 2|33.0| 0| 0| 13.0| 0| 1| 0.0| 0.0| 0.0|
 0| 3|31.0| 1| 0| 18.0| 1| 0| 1.0| 0.0| 2.0|
 1| 3|36.0| 0| 0| 7.225| 0| 1| 1.0| 1.0| 2.0|
+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+
only showing top 20 rows

Let's put all features into a vector

In [74]:
feature = VectorAssembler(inputCols=titanic_df.columns[1:],outputCol="features")
feature_vector= feature.transform(titanic_df)

In [75]:
feature_vector.show()

+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+--------------------+
Survived|Pclass| Age|SibSp|Parch| Fare|Family_Size|Alone|Sex_index|Embarked_index|Initial_index| features|
+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+--------------------+
 0| 3|22.0| 1| 0| 7.25| 1| 0| 0.0| 0.0| 0.0|(10,[0,1,2,4,5],[...|
 1| 1|38.0| 1| 0|71.2833| 1| 0| 1.0| 1.0| 2.0|[1.0,38.0,1.0,0.0...|
 1| 3|26.0| 0| 0| 7.925| 0| 1| 1.0| 0.0| 1.0|[3.0,26.0,0.0,0.0...|
 1| 1|35.0| 1| 0| 53.1| 1| 0| 1.0| 0.0| 2.0|[1.0,35.0,1.0,0.0...|
 0| 3|35.0| 0| 0| 8.05| 0| 1| 0.0| 0.0| 0.0|(10,[0,1,4,6],[3....|
 0| 3|33.0| 0| 0| 8.4583| 0| 1| 0.0| 2.0| 0.0|(10,[0,1,4,6,8],[...|
 0| 1|54.0| 0| 0|51.8625| 0| 1| 0.0| 0.0| 0.0|(10,[0,1,4,6],[1....|
 0| 3| 2.0| 3| 1| 21.075| 4| 0| 0.0| 0.0| 3.0|[3.0,2.0,3.0,1.0,...|
 1| 3|27.0| 0| 2|11.1333| 2| 0| 1.0| 0.0| 2.0|[3.0,27.0,0.0,2.0...|
 1| 2|14.0| 1| 0|30.0708| 1| 0| 1.0| 1.0| 2.0|[2.0,14.0,1.0,0.0...|
 1| 3| 4.0| 1| 1| 16.7| 2| 0| 1.0| 0.0| 1.0|[3.0,4.0,1.0,1.0,...|
 1| 1|58.0| 0| 0| 26.55| 0| 1| 1.0| 0.0| 1.0|[1.0,58.0,0.0,0.0...|
 0| 3|20.0| 0| 0| 8.05| 0| 1| 0.0| 0.0| 0.0|(10,[0,1,4,6],[3....|
 0| 3|39.0| 1| 5| 31.275| 6| 0| 0.0| 0.0| 0.0|[3.0,39.0,1.0,5.0...|
 0| 3|14.0| 0| 0| 7.8542| 0| 1| 1.0| 0.0| 1.0|[3.0,14.0,0.0,0.0...|
 1| 2|55.0| 0| 0| 16.0| 0| 1| 1.0| 0.0| 2.0|[2.0,55.0,0.0,0.0...|
 0| 3| 2.0| 4| 1| 29.125| 5| 0| 0.0| 2.0| 3.0|[3.0,2.0,4.0,1.0,...|
 1| 2|33.0| 0| 0| 13.0| 0| 1| 0.0| 0.0| 0.0|(10,[0,1,4,6],[2....|
 0| 3|31.0| 1| 0| 18.0| 1| 0| 1.0| 0.0| 2.0|[3.0,31.0,1.0,0.0...|
 1| 3|36.0| 0| 0| 7.225| 0| 1| 1.0| 1.0| 2.0|[3.0,36.0,0.0,0.0...|
+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+--------------------+
only showing top 20 rows

Now that the data is all set, let's split it into training and test. I'll be using a 80%, 80% split.

In [77]:
(trainingData, testData) = feature_vector.randomSplit([0.8, 0.2],seed = 11)

### Modelling

Here is the list of few Classification Algorithms from Spark ML 

LogisticRegression

DecisionTreeClassifier

RandomForestClassifier

Gradient-boosted tree classifier

NaiveBayes

Support Vector Machine

LogisticRegression

In [81]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="Survived", featuresCol="features")
#Training algo
lrModel = lr.fit(trainingData)
lr_prediction = lrModel.transform(testData)
lr_prediction.select("prediction", "Survived", "features").show()
evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")

+----------+--------+--------------------+
prediction|Survived| features|
+----------+--------+--------------------+
 0.0| 0|[1.0,19.0,3.0,2.0...|
 1.0| 0|[1.0,27.0,0.0,2.0...|
 0.0| 0|(10,[0,1,4,6],[1....|
 1.0| 0|[1.0,28.0,1.0,0.0...|
 0.0| 0|(10,[0,1,4,6],[1....|
 0.0| 0|(10,[0,1,4,6,8],[...|
 1.0| 0|(10,[0,1,3,4,5],[...|
 0.0| 0|(10,[0,1,6],[1.0,...|
 0.0| 0|(10,[0,1,4,6,8],[...|
 0.0| 0|(10,[0,1,2,4,5],[...|
 0.0| 0|[1.0,51.0,0.0,1.0...|
 0.0| 0|(10,[0,1,4,6,8],[...|
 0.0| 0|(10,[0,1,4,6,8],[...|
 0.0| 0|(10,[0,1,4,6],[2....|
 0.0| 0|(10,[0,1,4,6],[2....|
 0.0| 0|(10,[0,1,4,6],[2....|
 0.0| 0|(10,[0,1,2,4,5],[...|
 0.0| 0|(10,[0,1,2,4,5],[...|
 0.0| 0|(10,[0,1,2,4,5],[...|
 0.0| 0|(10,[0,1,4,6],[2....|
+----------+--------+--------------------+
only showing top 20 rows

Evaluate how well is LogisticRegression doing

In [83]:
lr_accuracy = evaluator.evaluate(lr_prediction)
print("Accuracy of LogisticRegression is = %g"% (lr_accuracy))
print("Test Error of LogisticRegression = %g " % (1.0 - lr_accuracy))

Accuracy of LogisticRegression is = 0.836257
Test Error of LogisticRegression = 0.163743

DecisionTreeClassifier

In [85]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol="Survived", featuresCol="features")
dt_model = dt.fit(trainingData)
dt_prediction = dt_model.transform(testData)
dt_prediction.select("prediction", "Survived", "features").show()


+----------+--------+--------------------+
prediction|Survived| features|
+----------+--------+--------------------+
 0.0| 0|[1.0,19.0,3.0,2.0...|
 0.0| 0|[1.0,27.0,0.0,2.0...|
 0.0| 0|(10,[0,1,4,6],[1....|
 0.0| 0|[1.0,28.0,1.0,0.0...|
 0.0| 0|(10,[0,1,4,6],[1....|
 0.0| 0|(10,[0,1,4,6,8],[...|
 0.0| 0|(10,[0,1,3,4,5],[...|
 0.0| 0|(10,[0,1,6],[1.0,...|
 0.0| 0|(10,[0,1,4,6,8],[...|
 0.0| 0|(10,[0,1,2,4,5],[...|
 0.0| 0|[1.0,51.0,0.0,1.0...|
 0.0| 0|(10,[0,1,4,6,8],[...|
 0.0| 0|(10,[0,1,4,6,8],[...|
 0.0| 0|(10,[0,1,4,6],[2....|
 1.0| 0|(10,[0,1,4,6],[2....|
 0.0| 0|(10,[0,1,4,6],[2....|
 0.0| 0|(10,[0,1,2,4,5],[...|
 0.0| 0|(10,[0,1,2,4,5],[...|
 0.0| 0|(10,[0,1,2,4,5],[...|
 0.0| 0|(10,[0,1,4,6],[2....|
+----------+--------+--------------------+
only showing top 20 rows

Accuracy of DecisionTreeClassifier is = 0.807018
Test Error of DecisionTreeClassifier = 0.192982

Evaluate how well is DecisionTreeClassifier doing

In [87]:
dt_accuracy = evaluator.evaluate(dt_prediction)
print("Accuracy of DecisionTreeClassifier is = %g"% (dt_accuracy))
print("Test Error of DecisionTreeClassifier = %g " % (1.0 - dt_accuracy))


Accuracy of DecisionTreeClassifier is = 0.807018
Test Error of DecisionTreeClassifier = 0.192982

RandomForestClassifier

In [89]:
from pyspark.ml.classification import RandomForestClassifier
rf = DecisionTreeClassifier(labelCol="Survived", featuresCol="features")
rf_model = rf.fit(trainingData)
rf_prediction = rf_model.transform(testData)
rf_prediction.select("prediction", "Survived", "features").show()

+----------+--------+--------------------+
prediction|Survived| features|
+----------+--------+--------------------+
 0.0| 0|[1.0,19.0,3.0,2.0...|
 0.0| 0|[1.0,27.0,0.0,2.0...|
 0.0| 0|(10,[0,1,4,6],[1....|
 0.0| 0|[1.0,28.0,1.0,0.0...|
 0.0| 0|(10,[0,1,4,6],[1....|
 0.0| 0|(10,[0,1,4,6,8],[...|
 0.0| 0|(10,[0,1,3,4,5],[...|
 0.0| 0|(10,[0,1,6],[1.0,...|
 0.0| 0|(10,[0,1,4,6,8],[...|
 0.0| 0|(10,[0,1,2,4,5],[...|
 0.0| 0|[1.0,51.0,0.0,1.0...|
 0.0| 0|(10,[0,1,4,6,8],[...|
 0.0| 0|(10,[0,1,4,6,8],[...|
 0.0| 0|(10,[0,1,4,6],[2....|
 1.0| 0|(10,[0,1,4,6],[2....|
 0.0| 0|(10,[0,1,4,6],[2....|
 0.0| 0|(10,[0,1,2,4,5],[...|
 0.0| 0|(10,[0,1,2,4,5],[...|
 0.0| 0|(10,[0,1,2,4,5],[...|
 0.0| 0|(10,[0,1,4,6],[2....|
+----------+--------+--------------------+
only showing top 20 rows

Evaluate how well is RandomForestClassifier doing

In [91]:
rf_accuracy = evaluator.evaluate(rf_prediction)
print("Accuracy of RandomForestClassifier is = %g"% (rf_accuracy))
print("Test Error of RandomForestClassifier  = %g " % (1.0 - rf_accuracy))

Accuracy of RandomForestClassifier is = 0.807018
Test Error of RandomForestClassifier = 0.192982

Gradient-boosted tree classifier

In [93]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(labelCol="Survived", featuresCol="features",maxIter=10)
gbt_model = gbt.fit(trainingData)
gbt_prediction = gbt_model.transform(testData)
gbt_prediction.select("prediction", "Survived", "features").show()


+----------+--------+--------------------+
prediction|Survived| features|
+----------+--------+--------------------+
 0.0| 0|[1.0,19.0,3.0,2.0...|
 1.0| 0|[1.0,27.0,0.0,2.0...|
 0.0| 0|(10,[0,1,4,6],[1....|
 1.0| 0|[1.0,28.0,1.0,0.0...|
 1.0| 0|(10,[0,1,4,6],[1....|
 0.0| 0|(10,[0,1,4,6,8],[...|
 0.0| 0|(10,[0,1,3,4,5],[...|
 0.0| 0|(10,[0,1,6],[1.0,...|
 0.0| 0|(10,[0,1,4,6,8],[...|
 0.0| 0|(10,[0,1,2,4,5],[...|
 0.0| 0|[1.0,51.0,0.0,1.0...|
 0.0| 0|(10,[0,1,4,6,8],[...|
 0.0| 0|(10,[0,1,4,6,8],[...|
 0.0| 0|(10,[0,1,4,6],[2....|
 0.0| 0|(10,[0,1,4,6],[2....|
 0.0| 0|(10,[0,1,4,6],[2....|
 0.0| 0|(10,[0,1,2,4,5],[...|
 0.0| 0|(10,[0,1,2,4,5],[...|
 0.0| 0|(10,[0,1,2,4,5],[...|
 0.0| 0|(10,[0,1,4,6],[2....|
+----------+--------+--------------------+
only showing top 20 rows

Accuracy of Gradient-boosted tree classifie is = 0.824561
Test Error of Gradient-boosted tree classifie 0.175439

Evaluate how well is Gradient-boosted doing

In [95]:
gbt_accuracy = evaluator.evaluate(gbt_prediction)
print("Accuracy of Gradient-boosted tree classifie is = %g"% (gbt_accuracy))
print("Test Error of Gradient-boosted tree classifie %g"% (1.0 - gbt_accuracy))

Accuracy of Gradient-boosted tree classifie is = 0.824561
Test Error of Gradient-boosted tree classifie 0.175439

Evaluate how well is DecisionTreeClassifier doing

NaiveBayes

In [98]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(labelCol="Survived", featuresCol="features")
nb_model = nb.fit(trainingData)
nb_prediction = nb_model.transform(testData)
nb_prediction.select("prediction", "Survived", "features").show()


Evaluate how well is NaiveBayes doing

In [100]:
nb_accuracy = evaluator.evaluate(nb_prediction)
print("Accuracy of NaiveBayes is  = %g"% (nb_accuracy))
print("Test Error of NaiveBayes  = %g " % (1.0 - nb_accuracy))

Accuracy of NaiveBayes is = 0.695906
Test Error of NaiveBayes = 0.304094

Support Vector Machine

In [102]:
from pyspark.ml.classification import LinearSVC
svm = LinearSVC(labelCol="Survived", featuresCol="features")
svm_model = svm.fit(trainingData)
svm_prediction = svm_model.transform(testData)
svm_prediction.select("prediction", "Survived", "features").show()


Evaluate how well is Support Vector Machine doing

In [104]:
svm_accuracy = evaluator.evaluate(svm_prediction)
print("Accuracy of Support Vector Machine is = %g"% (svm_accuracy))
print("Test Error of Support Vector Machine = %g " % (1.0 - svm_accuracy))

Accuracy of Support Vector Machine is = 0.836257
Test Error of Support Vector Machine = 0.163743

How to increase accuracy of a model ?
  * Add new features or drop existing features and train model
  * Tune ML algorith (https://spark.apache.org/docs/latest/ml-tuning.html)

### Reference

https://spark.apache.org/docs/latest/ml-classification-regression.html